In [1]:
%load_ext autoreload
%autoreload 2
%aimport RNN_model, RNN_apply_ind

# MSE

In [2]:
import RNN_apply_ind, os, json, argparse

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'
apply_file_name = '\Schedule_Monthly_Summary_2023Q123.csv'
# Load parameters from the JSON file.
if not os.path.exists('parameters.json'):
    print("parameters.json does not exist, Find the file and put it in the same folder as this file")
with open('parameters.json', 'r') as f:
    args = argparse.Namespace(**json.load(f))

RNN_apply_ind.main_apply(args, folder_path, seats_file_name, perf_file_name, apply_file_name)

Using the provided arguments.
-------- Start ----------
Will use cpu as device
Missing airports database loaded.
Major hubs database loaded.
Dimension mapping loaded.
Applying data does not exist, will create it
The data is cleaned.
The geo features are assigned.
The origin, destination and airline are encoded.
The airports coordinates are calculated.
The airline features are assigned.
The pandamic features are assigned.
The competitors are calculated.
The market share is calculated.
The market size is calculated.
The test data is created.
Original data loaded.
Main scaler rebuilt.
Seat scaler rebuilt.
Apply data scaled.
Validation data scaled.
Date features created.
Validation data prepared.
Model loaded
Evaluating: batch 100 of 664
Evaluating: batch 200 of 664
Evaluating: batch 300 of 664
Evaluating: batch 400 of 664
Evaluating: batch 500 of 664
Evaluating: batch 600 of 664
           MAE          RMSE       MAPE  std       <1%       <5%      <10%  \
0  4260.858398   6365.730957  28.

# GLS

In [ ]:
import RNN_apply_ind, os, json, argparse

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'
apply_file_name = '\Schedule_Monthly_Summary_2023Q1234.csv'
# Load parameters from the JSON file.
if not os.path.exists('parameters.json'):
    print("parameters.json does not exist, Find the file and put it in the same folder as this file")
with open('parameters.json', 'r') as f:
    args = argparse.Namespace(**json.load(f))

RNN_apply_ind.main_apply(args, folder_path, seats_file_name, perf_file_name, apply_file_name)